---
title: Model to Correct Homophone Misue
author: Marion Bauman
---


For our natural language processing model, we will be building a tool that corrects homophone misuse.

Our model will compute the likelihood that a given word is correct, and if it is not, it will suggest a replacement word. This will be tested on a corpus of data including some intentional homophone misuse.

In [28]:
from transformers import BertForMaskedLM, BertTokenizer, AdamW, get_linear_schedule_with_warmup

# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
model = BertForMaskedLM.from_pretrained('bert-base-cased')

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'bert.pooler.dense.weight', 'bert.pooler.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [30]:
# Chat GPT Prompt: Give me a list of lists in python of 100 sets of homophones
homophones_list = [
    ["ate", "eight"],
    ["bare", "bear"],
    ["brake", "break"],
    ["capital", "capitol"],
    ["cell", "sell"],
    ["cite", "site", "sight"],
    ["complement", "compliment"],
    ["desert", "dessert"],
    ["die", "dye"],
    ["flour", "flower"],
    ["hear", "here"],
    ["hour", "our"],
    ["knight", "night"],
    ["know", "no"],
    ["mail", "male"],
    ["meat", "meet"],
    ["morning", "mourning"],
    ["one", "won"],
    ["pair", "pear"],
    ["peace", "piece"],
    ["principal", "principle"],
    ["rain", "reign", "rein"],
    ["right", "write"],
    ["sea", "see"],
    ["serial", "cereal"],
    ["sole", "soul"],
    ["stationary", "stationery"],
    ["tail", "tale"],
    ["threw", "through"],
    ["to", "too", "two"],
    ["weather", "whether"],
    ["week", "weak"],
    ["wear", "where"],
    ["which", "witch"],
    ["your", "you're"],
    ["allowed", "aloud"],
    ["board", "bored"],
    ["brake", "break"],
    ["capital", "capitol"],
    ["compliment", "complement"],
    ["desert", "dessert"],
    ["dual", "duel"],
    ["fair", "fare"],
    ["genre", "jinja"],
    ["hare", "hair"],
    ["here", "hear"],
    ["hoard", "horde"],
    ["loan", "lone"],
    ["pail", "pale"],
    ["peak", "peek", "pique"],
    ["profit", "prophet"],
    ["role", "roll"],
    ["root", "route"],
    ["sail", "sale"],
    ["scene", "seen"],
    ["serial", "cereal"],
    ["so", "sow"],
    ["stare", "stair"],
    ["steal", "steel"],
    ["their", "there", "they're"],
    ["throne", "thrown"],
    ["vain", "vein", "vane"],
    ["weak", "week"],
    ["wood", "would"],
    ["yew", "you"],
    ["bridal", "bridle"],
    ["cereal", "serial"],
    ["chord", "cord"],
    ["compliment", "complement"],
    ["dew", "due"],
    ["foul", "fowl"],
    ["grate", "great"],
    ["groan", "grown"],
    ["heal", "heel"],
    ["him", "hymn"],
    ["lay", "lie"],
    ["main", "mane"],
    ["marry", "merry"],
    ["mite", "might"],
    ["moose", "mousse"],
    ["mourn", "morn"],
    ["peace", "piece"],
    ["plum", "plumb"],
    ["pour", "pore"],
    ["rap", "wrap"],
    ["scene", "seen"],
    ["scent", "cent", "sent"],
    ["serial", "cereal"],
    ["shear", "sheer"],
    ["soar", "sore"],
    ["sow", "sew"],
    ["stake", "steak"],
    ["tide", "tied"],
    ["toe", "tow"],
    ["there", "their", "they're"],
    ["waist", "waste"],
    ["week", "weak"],
    ["write", "right", "rite"],
]

In [4]:
import pandas as pd

gutenberg_homophone_data = pd.read_csv('../data/gutenberg-homophone-errors.csv')

In [62]:
test_sentence = """He doesn't no how to code."""

In [63]:
tokenized_sentence = tokenizer.tokenize(test_sentence)
print(tokenized_sentence)

['He', 'doesn', "'", 't', 'no', 'how', 'to', 'code', '.']


In [64]:
# flatten the list of lists
homophones_list_flat = [item for sublist in homophones_list for item in sublist]

In [76]:
from transformers import pipeline

mask_filler = pipeline("fill-mask", "bert-base-cased")

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'bert.pooler.dense.weight', 'bert.pooler.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


PipelineException: No mask_token ([MASK]) found on the input

In [94]:
import torch
testy = test_sentence.split()
outs = []
for ts in range(0, len(testy)):
    ts_full = testy.copy()[:ts+1]
    print(ts_full)
    if ts_full[ts] in homophones_list_flat:
        print(ts_full[ts])
        ts_full[ts] = '[MASK]'
        ts_full = ' '.join(ts_full)
        print(ts_full)
        print(mask_filler(ts_full, top_k=3))
        outs.append(mask_filler(ts_full, top_k=5))

None


TypeError: 'NoneType' object is not subscriptable

In [93]:
outs

[[{'score': 0.9814466238021851,
   'token': 119,
   'token_str': '.',
   'sequence': "He doesn't."},
  {'score': 0.009747138246893883,
   'token': 132,
   'token_str': ';',
   'sequence': "He doesn't ;"},
  {'score': 0.00484459986910224,
   'token': 106,
   'token_str': '!',
   'sequence': "He doesn't!"},
  {'score': 0.003609249135479331,
   'token': 136,
   'token_str': '?',
   'sequence': "He doesn't?"},
  {'score': 0.0001153292105300352,
   'token': 1232,
   'token_str': '...',
   'sequence': "He doesn't..."}],
 [{'score': 0.7354617714881897,
   'token': 119,
   'token_str': '.',
   'sequence': "He doesn't no how."},
  {'score': 0.1854487657546997,
   'token': 136,
   'token_str': '?',
   'sequence': "He doesn't no how?"},
  {'score': 0.055269792675971985,
   'token': 106,
   'token_str': '!',
   'sequence': "He doesn't no how!"},
  {'score': 0.01971513405442238,
   'token': 132,
   'token_str': ';',
   'sequence': "He doesn't no how ;"},
  {'score': 0.0010626193834468722,
   'token